# vfio-pci

In [61]:
from fabrictestbed_extensions.fablib.fablib import FablibManager
fablib = FablibManager()

SLICE      = "dpdk-basicfwd"
SITE       = "STAR"
IMAGE      = "default_ubuntu_24"
CORES      = 8
RAM_GB     = 16
DISK_GB    = 100

slice = fablib.new_slice(name=SLICE)

nodes, ifaces = [], []

node1 = slice.add_node(name="h1", site=SITE, image=IMAGE, cores=CORES, ram=RAM_GB, disk=DISK_GB)

comp1 = node1.add_component(model="NIC_Basic", name="h1_basic_1")
iface1 = comp1.get_interfaces()[0]
iface1.set_mode("manual")
ifaces.append(iface1)

comp2 = node1.add_component(model="NIC_Basic", name="h1_basic_2")
iface2 = comp2.get_interfaces()[0]
iface2.set_mode("manual")
ifaces.append(iface2)

nodes.append(node1)

node2 = slice.add_node(name="h2", site=SITE, image=IMAGE, cores=CORES, ram=RAM_GB, disk=DISK_GB)

comp1 = node2.add_component(model="NIC_Basic", name="h2_basic_1")
iface1 = comp1.get_interfaces()[0]
iface1.set_mode("manual")
ifaces.append(iface1)

nodes.append(node2)

slice.add_l2network(name="dpdk_net", interfaces=ifaces)

slice.submit(wait=True)


Retry: 7, Time: 200 sec


ID,0bca4708-27be-4637-907c-8a049e80c081
Name,dpdk-basicfwd
Lease Expiration (UTC),2025-06-27 22:41:32 +0000
Lease Start (UTC),2025-06-26 22:41:32 +0000
Project ID,e34dd6bb-f899-499d-a7af-8dfa60559ecb
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
091dea7b-3d8a-4ea1-b54c-4e6d992efe3d,h1,8,16,100,default_ubuntu_24,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe5b:b1f8,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe5b:b1f8,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
d234e36e-e55e-41ad-95fb-3103c6d9b53a,h2,8,16,100,default_ubuntu_24,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe98:30fb,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe98:30fb,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
60b178ea-90ad-4177-8357-8278dbca3694,dpdk_net,L2,L2Bridge,STAR,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
h1-h1_basic_1-p1,p1,h1,dpdk_net,100,manual,,26:FA:F1:39:E4:40,enp7s0,enp7s0,fe80::24fa:f1ff:fe39:e440,4,HundredGigE0/0/0/9
h1-h1_basic_2-p1,p1,h1,dpdk_net,100,manual,,2A:51:D5:A4:20:1A,enp8s0,enp8s0,fe80::2851:d5ff:fea4:201a,4,HundredGigE0/0/0/9
h2-h2_basic_1-p1,p1,h2,dpdk_net,100,manual,,2A:E2:7E:C1:38:4C,enp7s0,enp7s0,fe80::28e2:7eff:fec1:384c,4,HundredGigE0/0/0/9



Time to print interfaces 209 seconds


'0bca4708-27be-4637-907c-8a049e80c081'

In [1]:
from fabrictestbed_extensions.fablib.attestable_switch import Attestable_Switch
from fabrictestbed_extensions.fablib.fablib import FablibManager

slice_name = "dpdk-basicfwd"

fablib = FablibManager()
slice = fablib.get_slice(name=slice_name)

nodes = []
nodes.append(slice.get_node(name="h1"))
nodes.append(slice.get_node(name="h2"))

In [60]:
# slice.delete()

In [ ]:
nodes[0].execute("sudo apt-get update && sudo apt-get install -y build-essential git meson ninja-build pkg-config libnuma-dev libibverbs-dev librdmacm-dev rdma-core linux-modules-extra-$(uname -r) python3-pyelftools")

In [3]:
nodes[0].execute("git clone https://github.com/DPDK/dpdk.git")

Cloning into 'dpdk'...


('', "Cloning into 'dpdk'...\n")

In [ ]:
nodes[0].execute("cd dpdk && meson setup build && ninja -C build && cd ..")

In [5]:
nodes[0].execute("echo 512 | sudo tee /proc/sys/vm/nr_hugepages && sudo mkdir -p /mnt/huge && echo \"nodev /mnt/huge hugetlbfs defaults 0 0\" | sudo tee -a /etc/fstab && sudo mount -t hugetlbfs nodev /mnt/huge")

512
nodev /mnt/huge hugetlbfs defaults 0 0


('512\nnodev /mnt/huge hugetlbfs defaults 0 0\n', '')

In [6]:
nodes[0].execute("sudo modprobe vfio-pci && echo \"vfio\" | sudo tee  /etc/modules-load.d/vfio.conf && echo \"vfio-pci\"  | sudo tee -a /etc/modules-load.d/vfio.conf")

vfio
vfio-pci


('vfio\nvfio-pci\n', '')

In [7]:
nodes[0].execute("sudo ./dpdk/usertools/dpdk-devbind.py -s")


Network devices using kernel driver
0000:03:00.0 'Virtio 1.0 network device 1041' if=enp3s0 drv=virtio-pci unused=vfio-pci *Active*
0000:07:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp7s0 drv=mlx5_core unused=vfio-pci 
0000:08:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp8s0 drv=mlx5_core unused=vfio-pci 

No 'Baseband' devices detected

No 'Crypto' devices detected

No 'DMA' devices detected

No 'Eventdev' devices detected

No 'Mempool' devices detected

No 'Compress' devices detected

Misc (rawdev) devices using kernel driver
0000:04:00.0 'Virtio 1.0 block device 1042' drv=virtio-pci unused=vfio-pci 

No 'Regex' devices detected

No 'ML' devices detected


("\nNetwork devices using kernel driver\n===================================\n0000:03:00.0 'Virtio 1.0 network device 1041' if=enp3s0 drv=virtio-pci unused=vfio-pci *Active*\n0000:07:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp7s0 drv=mlx5_core unused=vfio-pci \n0000:08:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp8s0 drv=mlx5_core unused=vfio-pci \n\nNo 'Baseband' devices detected\n==============================\n\nNo 'Crypto' devices detected\n============================\n\nNo 'DMA' devices detected\n=========================\n\nNo 'Eventdev' devices detected\n==============================\n\nNo 'Mempool' devices detected\n=============================\n\nNo 'Compress' devices detected\n==============================\n\nMisc (rawdev) devices using kernel driver\n=========================================\n0000:04:00.0 'Virtio 1.0 block device 1042' drv=virtio-pci unused=vfio-pci \n\nNo 'Regex' devices detected\n===========================\n\nNo '

In [8]:
nodes[0].execute("sudo ./dpdk/usertools/dpdk-devbind.py -u 0000:07:00.0 && sudo ./dpdk/usertools/dpdk-devbind.py --noiommu-mode -b vfio-pci 0000:07:00.0 && sudo ./dpdk/usertools/dpdk-devbind.py -u 0000:08:00.0 && sudo ./dpdk/usertools/dpdk-devbind.py --noiommu-mode -b vfio-pci 0000:08:00.0")

('Warning: enabling unsafe no IOMMU mode for VFIO drivers\n', '')

In [ ]:
nodes[0].execute("cd dpdk/build && meson configure -Dexamples=skeleton && ninja")

In [10]:
nodes[0].execute("sudo apt update -y && sudo apt install -y rdma-core ibverbs-providers")



Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Hit:4 http://security.ubuntu.com/ubuntu noble-security InRelease
Reading package lists...
Building dependency tree...
Reading state information...
217 packages can be upgraded. Run 'apt list --upgradable' to see them.


Reading package lists...
Building dependency tree...
Reading state information...
rdma-core is already the newest version (50.0-2ubuntu0.2).
ibverbs-providers is already the newest version (50.0-2ubuntu0.2).
ibverbs-providers set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 217 not upgraded.


("Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease\nHit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease\nHit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease\nHit:4 http://security.ubuntu.com/ubuntu noble-security InRelease\nReading package lists...\nBuilding dependency tree...\nReading state information...\n217 packages can be upgraded. Run 'apt list --upgradable' to see them.\nReading package lists...\nBuilding dependency tree...\nReading state information...\nrdma-core is already the newest version (50.0-2ubuntu0.2).\nibverbs-providers is already the newest version (50.0-2ubuntu0.2).\nibverbs-providers set to manually installed.\n0 upgraded, 0 newly installed, 0 to remove and 217 not upgraded.\n",
 '\nWARNING: apt does not have a stable CLI interface. Use with caution in scripts.\n\n\nWARNING: apt does not have a stable CLI interface. Use with caution in scripts.\n\n')

In [11]:
nodes[0].execute("cd dpdk && sudo ./build/examples/dpdk-skeleton -l 0-3 -n 4")

EAL: Detected CPU lcores: 8
EAL: Detected NUMA nodes: 1
EAL: Detected static linkage of DPDK
EAL: Multi-process socket /var/run/dpdk/rte/mp_socket
EAL: Selected IOVA mode 'PA'
EAL: VFIO support initialized
VIRTIO_INIT: eth_virtio_pci_init(): Failed to init PCI device
PCI_BUS: Requested device 0000:03:00.0 cannot be used
mlx5_common: Verbs device not found: 0000:07:00.0
mlx5_common: Failed to initialize device context.
PCI_BUS: Requested device 0000:07:00.0 cannot be used
mlx5_common: Verbs device not found: 0000:08:00.0
mlx5_common: Failed to initialize device context.
PCI_BUS: Requested device 0000:08:00.0 cannot be used
EAL: Error - exiting with code: 1
Error: number of ports must be even


('',
 "EAL: Detected CPU lcores: 8\nEAL: Detected NUMA nodes: 1\nEAL: Detected static linkage of DPDK\nEAL: Multi-process socket /var/run/dpdk/rte/mp_socket\nEAL: Selected IOVA mode 'PA'\nEAL: VFIO support initialized\nVIRTIO_INIT: eth_virtio_pci_init(): Failed to init PCI device\nPCI_BUS: Requested device 0000:03:00.0 cannot be used\nmlx5_common: Verbs device not found: 0000:07:00.0\nmlx5_common: Failed to initialize device context.\nPCI_BUS: Requested device 0000:07:00.0 cannot be used\nmlx5_common: Verbs device not found: 0000:08:00.0\nmlx5_common: Failed to initialize device context.\nPCI_BUS: Requested device 0000:08:00.0 cannot be used\nEAL: Error - exiting with code: 1\nError: number of ports must be even\n")

# mlx5

In [22]:
from fabrictestbed_extensions.fablib.fablib import FablibManager
fablib = FablibManager()

SLICE      = "dpdk-basicfwd"
SITE       = "STAR"
IMAGE      = "default_ubuntu_24"
CORES      = 8
RAM_GB     = 16
DISK_GB    = 100

slice = fablib.new_slice(name=SLICE)

nodes, ifaces = [], []

node1 = slice.add_node(name="h1", site=SITE, image=IMAGE, cores=CORES, ram=RAM_GB, disk=DISK_GB)

comp1 = node1.add_component(model="NIC_Basic", name="h1_basic_1")
iface1 = comp1.get_interfaces()[0]
iface1.set_mode("manual")
ifaces.append(iface1)

comp2 = node1.add_component(model="NIC_Basic", name="h1_basic_2")
iface2 = comp2.get_interfaces()[0]
iface2.set_mode("manual")
ifaces.append(iface2)

nodes.append(node1)

node2 = slice.add_node(name="h2", site=SITE, image=IMAGE, cores=CORES, ram=RAM_GB, disk=DISK_GB)

comp1 = node2.add_component(model="NIC_Basic", name="h2_basic_1")
iface1 = comp1.get_interfaces()[0]
iface1.set_mode("manual")
ifaces.append(iface1)

nodes.append(node2)

slice.add_l2network(name="dpdk_net", interfaces=ifaces)

slice.submit(wait=True)


Retry: 11, Time: 281 sec


ID,4acd25b5-5de5-4027-b518-21c1ea0c1c3a
Name,dpdk-basicfwd
Lease Expiration (UTC),2025-06-28 18:50:46 +0000
Lease Start (UTC),2025-06-27 18:50:46 +0000
Project ID,e34dd6bb-f899-499d-a7af-8dfa60559ecb
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
3828e9dc-5d02-4d67-94bd-d3c63192785f,h1,8,16,100,default_ubuntu_24,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe55:6b09,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe55:6b09,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a6671413-a174-4af7-b7e0-c21a8654ddcc,h2,8,16,100,default_ubuntu_24,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fead:78f2,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fead:78f2,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
1dfa453f-0abd-4aaa-8c49-20869888e7ab,dpdk_net,L2,L2Bridge,STAR,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
h1-h1_basic_2-p1,p1,h1,dpdk_net,100,manual,,06:66:50:E1:F5:36,enp7s0,enp7s0,fe80::466:50ff:fee1:f536,4,HundredGigE0/0/0/9
h1-h1_basic_1-p1,p1,h1,dpdk_net,100,manual,,26:BD:B4:2A:A6:BB,enp8s0,enp8s0,fe80::24bd:b4ff:fe2a:a6bb,4,HundredGigE0/0/0/9
h2-h2_basic_1-p1,p1,h2,dpdk_net,100,manual,,26:CC:22:5D:27:52,enp7s0,enp7s0,fe80::24cc:22ff:fe5d:2752,4,HundredGigE0/0/0/9



Time to print interfaces 294 seconds


'4acd25b5-5de5-4027-b518-21c1ea0c1c3a'

In [1]:
from fabrictestbed_extensions.fablib.attestable_switch import Attestable_Switch
from fabrictestbed_extensions.fablib.fablib import FablibManager

slice_name = "dpdk-basicfwd"

fablib = FablibManager()
slice = fablib.get_slice(name=slice_name)

nodes = []
nodes.append(slice.get_node(name="h1"))
nodes.append(slice.get_node(name="h2"))

In [21]:
# slice.delete()

In [2]:
h1, h2 = [n for n in slice.get_nodes()] 

In [ ]:
COMMON_SETUP = r"""
sudo apt-get update -y
sudo apt-get install -y git build-essential \
    meson ninja-build pkg-config libnuma-dev libelf-dev \
    python3-pyelftools libpcap-dev \
    rdma-core libibverbs-dev ibverbs-utils librdmacm-dev linux-modules-extra-$(uname -r) linux-headers-$(uname -r)

# 1 GiB of 2 MB huge-pages for DPDK
echo 512 | sudo tee /sys/kernel/mm/hugepages/hugepages-2048kB/nr_hugepages
sudo mkdir -p /mnt/huge
grep -q hugetlbfs /proc/mounts || sudo mount -t hugetlbfs nodev /mnt/huge

# Load Mellanox verbs stack
sudo modprobe -a ib_uverbs mlx5_core mlx5_ib

# Build + install the latest mainline DPDK with only the skeleton example
test -d dpdk || git clone --depth 1 https://github.com/DPDK/dpdk.git ~/dpdk
cd dpdk
meson setup build -Dexamples=skeleton --prefix=/usr/local
ninja -C build
sudo ninja -C build install
sudo ldconfig
"""

#
# 1. Provision both hosts
#
for node in (h1, h2):
    print(f"=== provisioning {node.get_name()} ===")
    node.execute(COMMON_SETUP)

In [4]:
print("--- h1 NICs visible to mlx5 PMD ---")
h1.execute("sudo /usr/local/bin/dpdk-devbind.py --status")

--- h1 NICs visible to mlx5 PMD ---

Network devices using kernel driver
0000:03:00.0 'Virtio 1.0 network device 1041' if=enp3s0 drv=virtio-pci unused= *Active*
0000:07:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp7s0 drv=mlx5_core unused= 
0000:08:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp8s0 drv=mlx5_core unused= 

No 'Baseband' devices detected

No 'Crypto' devices detected

No 'DMA' devices detected

No 'Eventdev' devices detected

No 'Mempool' devices detected

No 'Compress' devices detected

Misc (rawdev) devices using kernel driver
0000:04:00.0 'Virtio 1.0 block device 1042' drv=virtio-pci unused= 

No 'Regex' devices detected

No 'ML' devices detected


("\nNetwork devices using kernel driver\n===================================\n0000:03:00.0 'Virtio 1.0 network device 1041' if=enp3s0 drv=virtio-pci unused= *Active*\n0000:07:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp7s0 drv=mlx5_core unused= \n0000:08:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp8s0 drv=mlx5_core unused= \n\nNo 'Baseband' devices detected\n==============================\n\nNo 'Crypto' devices detected\n============================\n\nNo 'DMA' devices detected\n=========================\n\nNo 'Eventdev' devices detected\n==============================\n\nNo 'Mempool' devices detected\n=============================\n\nNo 'Compress' devices detected\n==============================\n\nMisc (rawdev) devices using kernel driver\n=========================================\n0000:04:00.0 'Virtio 1.0 block device 1042' drv=virtio-pci unused= \n\nNo 'Regex' devices detected\n===========================\n\nNo 'ML' devices detected\n==========

In [8]:
h1.execute("ls dpdk/build/examples")

dpdk-skeleton
dpdk-skeleton.p
skeleton


('dpdk-skeleton\ndpdk-skeleton.p\nskeleton\n', '')

In [ ]:
print("--- starting basicfwd on h1 ---")
h1.execute("""
sudo dpdk/build/examples/dpdk-skeleton \\
     -l 0-1 -n 4 -a 0000:07:00.0 -a 0000:08:00.0
""")

--- starting basicfwd on h1 ---
EAL: Detected CPU lcores: 8
EAL: Detected NUMA nodes: 1
EAL: Detected static linkage of DPDK
EAL: Multi-process socket /var/run/dpdk/rte/mp_socket
EAL: Selected IOVA mode 'PA'


In [15]:
h1.execute("sudo pkill -9 -f dpdk")

('', '')

# igb_uio

In [9]:
from fabrictestbed_extensions.fablib.fablib import FablibManager
fablib = FablibManager()

SLICE      = "dpdk-basicfwd"
SITE       = "STAR"
IMAGE      = "default_ubuntu_24"
CORES      = 8
RAM_GB     = 16
DISK_GB    = 100

slice = fablib.new_slice(name=SLICE)

nodes, ifaces = [], []

node1 = slice.add_node(name="h1", site=SITE, image=IMAGE, cores=CORES, ram=RAM_GB, disk=DISK_GB)

comp1 = node1.add_component(model="NIC_Basic", name="h1_basic_1")
iface1 = comp1.get_interfaces()[0]
iface1.set_mode("manual")
ifaces.append(iface1)

comp2 = node1.add_component(model="NIC_Basic", name="h1_basic_2")
iface2 = comp2.get_interfaces()[0]
iface2.set_mode("manual")
ifaces.append(iface2)

nodes.append(node1)

node2 = slice.add_node(name="h2", site=SITE, image=IMAGE, cores=CORES, ram=RAM_GB, disk=DISK_GB)

comp1 = node2.add_component(model="NIC_Basic", name="h2_basic_1")
iface1 = comp1.get_interfaces()[0]
iface1.set_mode("manual")
ifaces.append(iface1)

nodes.append(node2)

slice.add_l2network(name="dpdk_net", interfaces=ifaces)

slice.submit(wait=True)


Retry: 7, Time: 195 sec


ID,ddc64a4e-840a-40d8-85ea-e25eb5666f37
Name,dpdk-basicfwd
Lease Expiration (UTC),2025-06-28 21:20:57 +0000
Lease Start (UTC),2025-06-27 21:20:57 +0000
Project ID,e34dd6bb-f899-499d-a7af-8dfa60559ecb
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
1717e400-624c-426a-a60f-4e6937c1985f,h1,8,16,100,default_ubuntu_24,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe8c:a756,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe8c:a756,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
694ba44f-1606-4daa-b86e-cc4f9b23de62,h2,8,16,100,default_ubuntu_24,qcow2,star-w3.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fefc:6612,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fefc:6612,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
59f25351-52b0-4cbc-9098-8d2ccc64bb4b,dpdk_net,L2,L2Bridge,STAR,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
h1-h1_basic_1-p1,p1,h1,dpdk_net,100,manual,,06:66:50:E1:F5:36,enp7s0,enp7s0,fe80::466:50ff:fee1:f536,4,HundredGigE0/0/0/9
h1-h1_basic_2-p1,p1,h1,dpdk_net,100,manual,,26:BD:B4:2A:A6:BB,enp8s0,enp8s0,fe80::24bd:b4ff:fe2a:a6bb,4,HundredGigE0/0/0/9
h2-h2_basic_1-p1,p1,h2,dpdk_net,100,manual,,26:CC:22:5D:27:52,enp7s0,enp7s0,fe80::24cc:22ff:fe5d:2752,4,HundredGigE0/0/0/9



Time to print interfaces 211 seconds


'ddc64a4e-840a-40d8-85ea-e25eb5666f37'

In [44]:
from fabrictestbed_extensions.fablib.attestable_switch import Attestable_Switch
from fabrictestbed_extensions.fablib.fablib import FablibManager

slice_name = "dpdk-basicfwd"

fablib = FablibManager()
slice = fablib.get_slice(name=slice_name)

nodes = []
nodes.append(slice.get_node(name="h1"))
nodes.append(slice.get_node(name="h2"))

In [7]:
# slice.delete()

In [20]:
nodes[0].execute("sudo apt-get update -y && sudo apt-get install -y build-essential \
                        git \
                        libnuma-dev \
                        software-properties-common \
                        meson \
                        ninja-build")

Hit:1 http://security.ubuntu.com/ubuntu noble-security InRelease
Hit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease
Hit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease
Hit:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.10ubuntu1).
git is already the newest version (1:2.43.0-1ubuntu7.2).
git set to manually installed.
libnuma-dev is already the newest version (2.0.18-1build1).
software-properties-common is already the newest version (0.99.49.2).
software-properties-common set to manually installed.
The following NEW packages will be installed:
  meson ninja-build
0 upgraded, 2 newly installed, 0 to remove and 214 not upgraded.
Need to get 733 kB of archives.
After this operation, 4213 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com

('Hit:1 http://security.ubuntu.com/ubuntu noble-security InRelease\nHit:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease\nHit:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease\nHit:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease\nReading package lists...\nReading package lists...\nBuilding dependency tree...\nReading state information...\nbuild-essential is already the newest version (12.10ubuntu1).\ngit is already the newest version (1:2.43.0-1ubuntu7.2).\ngit set to manually installed.\nlibnuma-dev is already the newest version (2.0.18-1build1).\nsoftware-properties-common is already the newest version (0.99.49.2).\nsoftware-properties-common set to manually installed.\nThe following NEW packages will be installed:\n  meson ninja-build\n0 upgraded, 2 newly installed, 0 to remove and 214 not upgraded.\nNeed to get 733 kB of archives.\nAfter this operation, 4213 kB of additional disk space will be used.\nGet:1 http://nova.clou

In [ ]:
nodes[0].execute("test -e dpdk-24.11.2.tar.xz || wget https://fast.dpdk.org/rel/dpdk-24.11.2.tar.xz && \
                        tar xJf dpdk-24.11.2.tar.xz && \
                        cd dpdk-stable-24.11.2 && \
                        sudo meson setup build && \
                        cd build && \
                        sudo ninja && \
                        sudo meson install && \
                        ldconfig")

In [33]:
nodes[0].execute("sudo python3 dpdk-stable-24.11.2/usertools/dpdk-hugepages.py -p 1G --setup 4G")

('', '')

In [62]:
nodes[0].execute("test -d dpdk-kmods || git clone https://github.com/DPDK/dpdk-kmods.git && cd dpdk-kmods/linux/igb_uio && make && sudo modprobe uio && sudo insmod igb_uio.ko")

make -C /lib/modules/6.8.0-49-generic/build/ M=/home/ubuntu/dpdk-kmods/linux/igb_uio
make[1]: Entering directory '/usr/src/linux-headers-6.8.0-49-generic'
  The kernel was built by: x86_64-linux-gnu-gcc-13 (Ubuntu 13.2.0-23ubuntu4) 13.2.0
  You are using:           gcc-13 (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0
  CC [M]  /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.o
  MODPOST /home/ubuntu/dpdk-kmods/linux/igb_uio/Module.symvers
  CC [M]  /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.mod.o
  LD [M]  /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.ko
  BTF [M] /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.ko
make[1]: Leaving directory '/usr/src/linux-headers-6.8.0-49-generic'
Skipping BTF generation for /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.ko due to unavailability of vmlinux


("make -C /lib/modules/6.8.0-49-generic/build/ M=/home/ubuntu/dpdk-kmods/linux/igb_uio\nmake[1]: Entering directory '/usr/src/linux-headers-6.8.0-49-generic'\n  CC [M]  /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.o\n  MODPOST /home/ubuntu/dpdk-kmods/linux/igb_uio/Module.symvers\n  CC [M]  /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.mod.o\n  LD [M]  /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.ko\n  BTF [M] /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.ko\nmake[1]: Leaving directory '/usr/src/linux-headers-6.8.0-49-generic'\n",
 'warning: the compiler differs from the one used to build the kernel\n  The kernel was built by: x86_64-linux-gnu-gcc-13 (Ubuntu 13.2.0-23ubuntu4) 13.2.0\n  You are using:           gcc-13 (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0\nSkipping BTF generation for /home/ubuntu/dpdk-kmods/linux/igb_uio/igb_uio.ko due to unavailability of vmlinux\n')

In [63]:
nodes[0].execute("sudo modprobe uio && sudo modprobe igb_uio && lsmod | grep -E 'igb_uio|uio'")

igb_uio                24576  0
uio                    28672  1 igb_uio


('igb_uio                24576  0\nuio                    28672  1 igb_uio\n',
 '')

In [64]:
nodes[0].execute("python3 dpdk-stable-24.11.2/usertools/dpdk-devbind.py -s")


Network devices using kernel driver
0000:03:00.0 'Virtio 1.0 network device 1041' if=enp3s0 drv=virtio-pci unused=igb_uio *Active*
0000:07:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp7s0 drv=mlx5_core unused=igb_uio 
0000:08:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp8s0 drv=mlx5_core unused=igb_uio 

No 'Baseband' devices detected

No 'Crypto' devices detected

No 'DMA' devices detected

No 'Eventdev' devices detected

No 'Mempool' devices detected

No 'Compress' devices detected

Misc (rawdev) devices using kernel driver
0000:04:00.0 'Virtio 1.0 block device 1042' drv=virtio-pci unused=igb_uio 

No 'Regex' devices detected

No 'ML' devices detected


("\nNetwork devices using kernel driver\n===================================\n0000:03:00.0 'Virtio 1.0 network device 1041' if=enp3s0 drv=virtio-pci unused=igb_uio *Active*\n0000:07:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp7s0 drv=mlx5_core unused=igb_uio \n0000:08:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' if=enp8s0 drv=mlx5_core unused=igb_uio \n\nNo 'Baseband' devices detected\n==============================\n\nNo 'Crypto' devices detected\n============================\n\nNo 'DMA' devices detected\n=========================\n\nNo 'Eventdev' devices detected\n==============================\n\nNo 'Mempool' devices detected\n=============================\n\nNo 'Compress' devices detected\n==============================\n\nMisc (rawdev) devices using kernel driver\n=========================================\n0000:04:00.0 'Virtio 1.0 block device 1042' drv=virtio-pci unused=igb_uio \n\nNo 'Regex' devices detected\n===========================\n\nNo 'ML' 

In [67]:
nodes[0].execute("sudo python3 dpdk-stable-24.11.2/usertools/dpdk-devbind.py -b igb_uio 0000:07:00.0")

('', '')

In [68]:
nodes[0].execute("sudo python3 dpdk-stable-24.11.2/usertools/dpdk-devbind.py -b igb_uio 0000:08:00.0")

('', '')

In [76]:
nodes[0].execute("python3 dpdk-stable-24.11.2/usertools/dpdk-devbind.py -s")


Network devices using DPDK-compatible driver
0000:07:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' drv=igb_uio unused=mlx5_core
0000:08:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' drv=igb_uio unused=mlx5_core

Network devices using kernel driver
0000:03:00.0 'Virtio 1.0 network device 1041' if=enp3s0 drv=virtio-pci unused=igb_uio *Active*

No 'Baseband' devices detected

No 'Crypto' devices detected

No 'DMA' devices detected

No 'Eventdev' devices detected

No 'Mempool' devices detected

No 'Compress' devices detected

Misc (rawdev) devices using kernel driver
0000:04:00.0 'Virtio 1.0 block device 1042' drv=virtio-pci unused=igb_uio 

No 'Regex' devices detected

No 'ML' devices detected


("\nNetwork devices using DPDK-compatible driver\n============================================\n0000:07:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' drv=igb_uio unused=mlx5_core\n0000:08:00.0 'MT28908 Family [ConnectX-6 Virtual Function] 101c' drv=igb_uio unused=mlx5_core\n\nNetwork devices using kernel driver\n===================================\n0000:03:00.0 'Virtio 1.0 network device 1041' if=enp3s0 drv=virtio-pci unused=igb_uio *Active*\n\nNo 'Baseband' devices detected\n==============================\n\nNo 'Crypto' devices detected\n============================\n\nNo 'DMA' devices detected\n=========================\n\nNo 'Eventdev' devices detected\n==============================\n\nNo 'Mempool' devices detected\n=============================\n\nNo 'Compress' devices detected\n==============================\n\nMisc (rawdev) devices using kernel driver\n=========================================\n0000:04:00.0 'Virtio 1.0 block device 1042' drv=virtio-pci unused=igb_uio 

In [ ]:
basicfwd_setup = r"""
test -d dpdk || git clone --depth 1 https://github.com/DPDK/dpdk.git ~/dpdk
cd dpdk
meson setup build -Dexamples=skeleton --prefix=/usr/local
ninja -C build
sudo ninja -C build install
sudo ldconfig
"""

nodes[0].execute(basicfwd_setup)

In [77]:
nodes[0].execute("""
sudo dpdk/build/examples/dpdk-skeleton \\
     -l 1 -n 4 -a 0000:07:00.0 -a 0000:08:00.0
""")

EAL: Detected CPU lcores: 8
EAL: Detected NUMA nodes: 1
EAL: Detected static linkage of DPDK
EAL: Multi-process socket /var/run/dpdk/rte/mp_socket
EAL: Selected IOVA mode 'PA'
EAL: Error - exiting with code: 1
Error: number of ports must be even


('',
 "EAL: Detected CPU lcores: 8\nEAL: Detected NUMA nodes: 1\nEAL: Detected static linkage of DPDK\nEAL: Multi-process socket /var/run/dpdk/rte/mp_socket\nEAL: Selected IOVA mode 'PA'\nEAL: Error - exiting with code: 1\nError: number of ports must be even\n")